**Número de grupo:** 25

**Nombre de los integrantes del grupo:**
- David Bugoi
- Daniela Alejanda Córdova
- Erik Karlgren Domercq
 
Lab 11

# Práctica 1

> __Fecha de entrega: 11 de abril de 2021__


## Parte 2: similitud semántica

Una de las grandes ventajas de las representaciones estructuradas es que podemos aprovechar su estructura para calcular similitudes semánticas entre las entidades. En esta ocasión vamos a cacular la similitud entre dos conceptos como:

$$Sim(A, B) = \frac{\delta(root, C)}{\delta(root, C) + \delta(C, A) + \delta(C, B)}$$

siendo:

- $\delta(X, Y)$ el __mínimo__ número de aristas que conecta A y B, siendo A más general que B.
- $C = LCS(A, B)$ el concepto más específico de la jerarquía que es más general que A y B (_least common subsummer_).

La idea tras esta similitud queda reflejada en la siguiente imagen:

<img src="sim.png" alt="Similitud" style="width: 300px;"/>

En la práctica pueden existir distintos conceptos C que cumplen la definición de _least common subsummer_ de A y B por lo que es necesario definir cuál de ellos vamos a utilizar. En nuestro caso seleccionaremos __uno de los que maximiza el valor de similitud__. 

### 1) Obtener la taxonomía con la que vamos a trabajar

Utiliza el [punto el acceso](https://query.wikidata.org/) SPARQL de Wikidata para ejecutar una consulta que devuelva todos los pares de entidades $(x, y)$ tal que $x$ es subconcepto directo de $y$ y ambos son un tipos de [instrumentos musicales (Q34379)](https://www.wikidata.org/wiki/Q34379). Debes recuperar tantos las URIs de la entidades como sus etiquetas.

Escribe en la siguiente celda la consulta que has utilizado comentada adecuadamente.

A continuación descarga todas las respuestas en formato _Archivo JSON_ y guardalo en el mismo directorio de la práctica.

_Nota: en el momento de realizar esta práctica obtuve 4727 resultados pero el número puede variar al ser Wikidata una base de conocimiento dinámica._

### 2) Cargar la taxonomía en memoria

Vamos a cargar la taxonomía de clases en memoria para poder operar con ella. Representaremos la jerarquía de clases mediante las siguientes estructuras:

- Un diccionario que asocia a cada identificador su etiqueta (por ejemplo 'Q34379' -> 'musical instrument')
- Un diccionario que asocia cada clase con sus subclases directas (por ejemplo 'Q17172850' -> {'Q101436564', 'Q1067089', 'Q186506', 'Q210970', 'Q223166', ...}, )
- Un diccionario que asocia cada clase con sus superclases directas (por ejemplo 'Q5994' -> {'Q3152898', 'Q4951628', 'Q52954'})

Tienes libertad para elegir cómo quieres representar la taxonomía en Python:

- Puedes usar una clase. En ese caso tendrás que ir añadiendo métodos a la clase para completar cada uno de los apartados de la práctica. Escribe el código de la clase en una única celda y utiliza los métodos que necesites en cada uno de los apartados.
- Puedes usar 3 variables globales para representar la taxonomía. En ese caso deberás escribir las operaciones como funciones en cada uno de los apartados de la práctica.

En cualquier caso recuerda documentar adecuadamente el código y trata de que sea sencillo de entender.

Crea una operación _load_ que reciba el nombre del fichero json y cargue el grafo en memoria usando las estructuras anteriores.

Recuerda que puedes cargar cualquier estructura en formato json usando el siguiente código:

```python
import json

with open(filename, encoding='utf-8') as f:
    data = json.load(f)
```

In [1]:
import json
filename = "query.json"
with open(filename, encoding='utf-8') as f:
    data = json.load(f)

# dicNombres = {} # nombre -> etiqueta (pueden haber problemas por duplicidades así que no lo usamos)
# Lo dejamos aquí para que se entienda la elección del nombre del diccionario 'dicNombresInversa'

dicNombresInversa ={} # etiqueta -> nombre
dicSubclases = {} # clase -> subclases
dicSuperclases = {} # clase -> superclases

for i in data:
    etiquetaX = i['x'].replace('http://www.wikidata.org/entity/', '')
    etiquetaY = i['y'].replace('http://www.wikidata.org/entity/', '')
    # dicNombres[i['xLabel']] = etiquetaX
    dicNombresInversa[etiquetaX] = i['xLabel']
    # dicNombres[i['yLabel']] = etiquetaY
    dicNombresInversa[etiquetaY] = i['yLabel']


    if dicSubclases.__contains__(etiquetaY):
        dicSubclases[etiquetaY].append(etiquetaX)
    else:
        dicSubclases[etiquetaY] = [etiquetaX]
        
    if dicSuperclases.__contains__(etiquetaX):
        dicSuperclases[etiquetaX].append(etiquetaY)
    else:
        dicSuperclases[etiquetaX] = [etiquetaY]

### 3) Imprimir un subárbol de la taxonomía

Crea una operación _print_tree_ que imprimir la jerarquía de clases a partir de un concepto y hasta un nivel de profundidad determinado.

Por ejemplo, a continuación podemos ver el principio de la jerarquía de [voces](https://www.wikidata.org/wiki/Q17172850) con 3 niveles de profundidad:

```
0 voz (Q17172850)
  1 operatic vocal (Q101436564)
  1 alto (Q6983813)
   2 mezzosoprano ligera (Q6012300)
   2 boy alto (Q53395277)
   2 alto castrato (Q53395016)
   2 contralto (Q37137)
  1 contralto (Q37137)
   2 contralto cómica (Q5785182)
   2 lyric contralto (Q54635214)
   2 Tenorino (Q6141663)
   2 contralto de coloratura (Q54635184)
   2 deep contralto (Q54635335)
   2 contralto dramática (Q5785183)
  1 bajo (Q27911)
   2 heavy acting bass (Q54636271)
   2 bajo profundo (Q2532487)
   2 bajo buffo (Q1002146)
   ...
```

Como ocurre en todas las grandes bases de conocimiento, dentro de Wikidata hay información que no ha sido bien introducida o está mal clasificada. ¿Puedes encontrar algún ejemplo concreto dentro de la jerarquía de instrumentos?

In [2]:
from collections import deque

# Devuelve un string con la clave y el nombre del instrumento
def key_to_string(key):
    return f'{dicNombresInversa[key]} ({key})'

# Imprime las subclases de 'etiqueta' que están a una profundidad menor o igual a 'prof'
def print_tree(etiqueta, prof):
    i = 0
    pila = []
    pila.append((i,etiqueta))
    
    while pila:
        i, actual = pila.pop()
        stringFinal = str(i) + " " + key_to_string(actual)
        string_length = len(stringFinal) + i  # will be adding 10 extra spaces
        string_revised = stringFinal.rjust(string_length)
        print(string_revised)
        
        if dicSubclases.__contains__(actual):
            for key in dicSubclases[actual]:
                if(i+1 < prof):
                    pila.append((i+1, key))


In [3]:
print_tree("Q6607", 4)

0 guitar (Q6607)
 1 Kent (guitar) (Q99902121)
 1 Q98457398 (Q98457398)
  2 requinto jarocho (Q7314870)
  2 Leona (Q6524983)
  2 jarana jarocha (Q1683429)
 1 guitar accessories (Q55185569)
 1 Brazilian guitars (Q48877774)
 1 requinto guitar (Q24196576)
 1 heavy metal guitar (Q23808566)
 1 G-sharp guitar (Q21279067)
 1 fretless guitar (Q20758461)
 1 Q20104968 (Q20104968)
 1 Q15725636 (Q15725636)
 1 six-string alto guitar (Q7532606)
 1 Leona (Q6524983)
 1 Halam (Q5891381)
 1 Eight-string guitar (Q5348940)
  2 Brahms guitar (Q4955628)
 1 Dean Soltero (Q5246477)
 1 Craviola (Q5182772)
 1 Fender Dodicaster (Q3742418)
 1 lyre-guitar (Q3675195)
 1 Touch Guitar (Q3532807)
 1 archtop guitar (Q3392409)
 1 Hawaiian Guitar (Q3120692)
 1 Burny (Q3069420)
 1 cuenca (Q3006756)
 1 Kabosy (Q2895785)
 1 Q2722253 (Q2722253)
 1 steel guitar (Q2617520)
  2 console steel guitar (Q5163252)
  2 pedal steel guitar (Q587027)
 1 Ten-string guitar (Q2569413)
 1 Vintage V100MRPGM Lemon Drop (Q2527208)
 1 Steinberge

### 4) Obtener los LCS

Crea una operación _lcs_ que devuelva todos los LCS de dos conceptos determinados. Recuerda que un concepto C es LCS(A, B) si es más general que ambos y no se puede especializar más sin dejar de serlo.

Para implementarlo seguramente te resulte útil tener otro método que devuelva todos los conceptos más generales que uno dado. _Pista: es fácil de implementar usando operaciones entre conjuntos_. 

Ejemplos:

```python
mezzosoprano dramática (Q6012297), mezzosoprano ligera (Q54634726), mezzosoprano (Q186506)
LCS('Q6012297', 'Q54634726') = {'Q186506'}

grave (Q5885030), mezzosoprano ligera (Q6012300), voz (Q17172850)
LCS('Q5885030', 'Q6012300') = {'Q17172850'}

tenor (Q27914)
LCS('Q27914', 'Q27914') = {'Q27914'}

viola eléctrica (Q15336282), bajo eléctrico (Q64166304), instrumento de cuerda (Q1798603), electrófono (Q105738), necked box lutes (Q55724840)
LCS('Q15336282', 'Q64166304') = {'Q55724840', 'Q105738', 'Q1798603'}
```

In [4]:
# Devuelve todos los conceptos más generales que 'e' y sus distancias respecto al concepto 'e'
def todosConceptosGenerales(e):
    sol = {}
    m = dicSuperclases[e]
    pila = []
    i = 1
    sol[e] = 0
    
    for key in m:
        pila.append((i, key))
        
    while pila:
        i, actual = pila.pop()
        if sol.__contains__(actual):
            sol[actual] = min (i, sol[actual])
        else:
            sol[actual]=i
        if dicSuperclases.__contains__(actual):
            for key in dicSuperclases[actual]:
                pila.append((i+1,key))
                
    return sol

Definimos una función `path(a,b)` que calcula el camino más corto desde el concepto `a` hasta el concepto `b`, suponiendo que `b` sea subclase de `a`. La habíamos definido para el apartado 5 y nos resultaba conveniente usarla para calcular el LCS porque este necesita estar lo más alejado posible de la raíz (instrumento musical).

In [5]:
# Busca el camino mínimo entre los conceptos A y B
def path(a,b):
    found = False            # indica si hemos encontrado 'b'
    nodes_to_visit = deque() # lista con los nodos por visitar
    previous = dict()        # diccionario con el nodo (o clave) anterior a cada nodo (o clave)
    
    nodes_to_visit.append(a) # primero visitamos el concepto 'a'
    previous[a] = ''
    
    # búsqueda en profundidad
    while nodes_to_visit and not found:
        current_node = nodes_to_visit.popleft()
        
        if current_node == b:
            found = True
            
        elif dicSubclases.__contains__(current_node):
            for item in dicSubclases[current_node]:
                nodes_to_visit.appendleft(item)
                previous[item] = current_node
    
    # reconstruimos la solución
    if not found:
        return list()
    else:
        path = deque()
        key = b
        while key != '':
            path.appendleft(key_to_string(key))
            key = previous[key]
        return list(path)

In [6]:
# Calcula el LCS (least common subsummer) de los conceptos A y B.
def LCS(A, B):
    dicA = todosConceptosGenerales(A)
    dicA_set = dicA.keys()
    dicB = todosConceptosGenerales(B)
    dicB_set = dicB.keys()
    difF = dicA_set & dicB_set

    valor_max=0
    valorFinal=set()
    
    for i in difF:
        a = len(path('Q34379',i)) # Camino de la raíz (instrumento musical) al concepto i
        
        # Nos quedamos solo con los LCS que están a la máxima distancia posible de la raíz
        if a > valor_max :
            valorFinal= {i}
            valor_max=a
        elif a == valor_max:
            valorFinal.add(i)
            valor_max=a
            
    return valorFinal

Comprobamos si los LCS que calcula nuestro algoritmo coinciden con los del enunciado de este apartado.

In [7]:
LCS('Q6012297', 'Q54634726')

{'Q186506'}

In [8]:
LCS('Q5885030', 'Q6012300')

{'Q17172850'}

In [9]:
LCS('Q27914', 'Q27914')

{'Q27914'}

Como dijimos en la definición de LCS(A,B), nos quedamos solo con los LCS que estén a la máxima distancia
posible de la raíz (instrumento musical). Es por eso que solo nos sale un resultado en esta consulta. A continuación vemos las superclases de Q15336282 y Q64166304 junto a la distancia (como número de clases) entre las superclases y la propia clase.

In [10]:
LCS('Q15336282', 'Q64166304')

{'Q55724840'}

In [11]:
todosConceptosGenerales("Q15336282")

{'Q15336282': 0,
 'Q80284': 1,
 'Q57050725': 2,
 'Q55724840': 3,
 'Q55724833': 4,
 'Q30038759': 5,
 'Q1808578': 6,
 'Q19588495': 7,
 'Q1051772': 5,
 'Q34379': 3,
 'Q192096': 3,
 'Q1798603': 4,
 'Q1327500': 1,
 'Q105738': 2}

In [12]:
todosConceptosGenerales("Q64166304")

{'Q64166304': 0,
 'Q78987': 1,
 'Q6607': 2,
 'Q57306162': 3,
 'Q55724840': 4,
 'Q55724833': 5,
 'Q30038759': 6,
 'Q1808578': 7,
 'Q19588495': 8,
 'Q1051772': 6,
 'Q34379': 3,
 'Q230262': 4,
 'Q1798603': 5,
 'Q55738960': 2,
 'Q50821977': 3,
 'Q105738': 4,
 'Q46185': 1,
 'Q810447': 2}

### 5) Obtener caminos mínimos

Crea una operación _path_ que calcule el camino mínimo entre dos conceptos A y B siendo A más o igual de general que B. Como la taxonomía no tiene ciclos puedes implementarlo como una búsqueda en profundidad. Ten en cuenta que los caminos sólo pueden contener conceptos más específicos o iguales a A y más generales o iguales a B.

Ejemplos:

```python
path('Q186506', 'Q54634726') = [mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

path('Q17172850', 'Q6012300') = [voz (Q17172850), alto (Q6983813), mezzosoprano ligera (Q6012300)]

path('Q27914', 'Q27914') = [tenor (Q27914)]

path('Q34379', 'Q105738') = [instrumento musical (Q34379), cordófono (Q1051772), composite chordophones (Q19588495), lutes (Q1808578), handle lutes (Q30038759), necked lutes (Q55724833), necked box lutes (Q55724840)]
 ```
La función `path` la habíamos definido al principio en este apartado, pero como nos fue útil para calcular el LCS la hemos definido finalmente en el apartado 4. A continuación mostramos algunos ejemplos.

In [13]:
path('Q186506', 'Q54634726') 

['mezzo-soprano (Q186506)', 'light mezzo-soprano (Q54634726)']

In [14]:
path('Q17172850', 'Q6012300')

['voice (Q17172850)', 'alto (Q6983813)', 'Q6012300 (Q6012300)']

In [15]:
path('Q27914', 'Q27914')

['tenor (Q27914)']

In [16]:
path('Q34379', 'Q55724840')

['musical instrument (Q34379)',
 'chordophone (Q1051772)',
 'composite chordophones (Q19588495)',
 'lutes (Q1808578)',
 'handle lutes (Q30038759)',
 'necked lutes (Q55724833)',
 'necked box lutes (Q55724840)']

In [17]:
path('Q34379', 'Q186506')

['musical instrument (Q34379)',
 'continuous-pitch instrument (Q98329515)',
 'voice (Q17172850)',
 'mezzo-soprano (Q186506)']

### 6) Calcular la similitud

Implementa una operación _similarity_ que calcule la similtud entre dos conceptos. Debe devolver tanto el valor númerico de similitud como los caminos desde la raiz al LCS y desde el LCS a cada uno de los dos conceptos.

Ten en cuenta que debes usar un LCS que maximice el valor de similitud. Si la información de Wikidata no ha cambiado, los valores de similitud deberían coincidir con los que aparecen en los ejemplos pero los caminos no tienen por qué. Y recuerda que no es lo mismo el números de aristas de un camino que el número de nodos del camino.

Ejemplos:

```python
similarity('Q6012297', 'Q54634726')
0.5
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano dramática (Q6012297)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q186506', 'Q54634726')
0.6666666666666666
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q27914', 'Q27914')
1.0
[instrumento musical (Q34379), voz (Q17172850), high voice (Q98116969), tenor (Q27914)]
[tenor (Q27914)]
[tenor (Q27914)]

similarity('Q76239', 'Q78987')
0.42857142857142855
[instrumento musical (Q34379), cordófono (Q1051772), instrumento de cuerda (Q1798603), instrumento de cuerda pulsada (Q230262)]
[instrumento de cuerda pulsada (Q230262), cítara (Q76239)]
[instrumento de cuerda pulsada (Q230262), plucked necked box lutes (Q57306162), guitarra (Q6607), guitarra eléctrica (Q78987)]
```

In [26]:
root = 'Q34379' # 'root' es instrumento musical

# Calcula la similitud entre 'a' y 'b'. Por defecto imprime no solo el valor de retorno sino también el camino
# de la raíz (instrumento musical) al LCS, y de este a 'a' y a 'b'.
def similarity(a,b,imprimir=True):
    lcs = list(LCS(a,b))[0]
    lcs_to_a_path = path(lcs, a)
    lcs_to_b_path = path(lcs, b)
    root_to_lcs_path = path(root, lcs)
    sim = 1;
    
    if a != b: # Para evitar dividir por 0
        sim = (len(root_to_lcs_path) - 1) / (len(root_to_lcs_path) + len(lcs_to_a_path) + len(lcs_to_b_path) - 3)
    
    if imprimir:
        print(sim)
        print(root_to_lcs_path)
        print(lcs_to_a_path)
        print(lcs_to_b_path)
        
    return sim

Las similitudes son ligeramente distintas a las del enunciado porque cuando hicimos nuestra consulta se había añadido otro concepto entre 'instrumento musical (Q34379)' y 'voz (Q17172850)': el Q98329515.

In [27]:
similarity('Q6012297', 'Q54634726')

0.6
['musical instrument (Q34379)', 'continuous-pitch instrument (Q98329515)', 'voice (Q17172850)', 'mezzo-soprano (Q186506)']
['mezzo-soprano (Q186506)', 'dramatic mezzo-soprano (Q6012297)']
['mezzo-soprano (Q186506)', 'light mezzo-soprano (Q54634726)']


0.6

In [29]:
similarity('Q186506', 'Q54634726')

0.75
['musical instrument (Q34379)', 'continuous-pitch instrument (Q98329515)', 'voice (Q17172850)', 'mezzo-soprano (Q186506)']
['mezzo-soprano (Q186506)']
['mezzo-soprano (Q186506)', 'light mezzo-soprano (Q54634726)']


0.75

In [30]:
similarity('Q27914', 'Q27914')

1
['musical instrument (Q34379)', 'continuous-pitch instrument (Q98329515)', 'voice (Q17172850)', 'high voice (Q98116969)', 'tenor (Q27914)']
['tenor (Q27914)']
['tenor (Q27914)']


1

In [31]:
similarity('Q76239', 'Q78987')

0.42857142857142855
['musical instrument (Q34379)', 'chordophone (Q1051772)', 'string instrument (Q1798603)', 'plucked string instrument (Q230262)']
['plucked string instrument (Q230262)', 'zither (Q76239)']
['plucked string instrument (Q230262)', 'plucked necked box lutes (Q57306162)', 'guitar (Q6607)', 'electric guitar (Q78987)']


0.42857142857142855

### 7) Análisis de las similitudes

Calcula la similitud 2 a 2 de los siguientes instrumentos y explica razonadamente si los valores obtenidos tienen sentido de acuerdo a tu intuición sobre si se parecen o no.

```
piano (Q5994), guitarra (Q6607), guitarra eléctrica (Q78987), flauta (Q11405), trompeta (Q8338)
```

In [20]:
instruments = ['Q5994', 'Q6607', 'Q78987', 'Q11405', 'Q8338']

for i in range(len(instruments)):
    for j in range(i+1, len(instruments)):
        strI = key_to_string(instruments[i])
        strJ = key_to_string(instruments[j])
        print(f'(Similitud) {strI} vs. {strJ}: {str(similarity(instruments[i], instruments[j], False))}')

(Similitud) piano (Q5994) vs. guitar (Q6607): 0.2727272727272727
(Similitud) piano (Q5994) vs. electric guitar (Q78987): 0.25
(Similitud) piano (Q5994) vs. flute (Q11405): 0.0
(Similitud) piano (Q5994) vs. trumpet (Q8338): 0.0
(Similitud) guitar (Q6607) vs. electric guitar (Q78987): 0.8888888888888888
(Similitud) guitar (Q6607) vs. flute (Q11405): 0.0
(Similitud) guitar (Q6607) vs. trumpet (Q8338): 0.0
(Similitud) electric guitar (Q78987) vs. flute (Q11405): 0.0
(Similitud) electric guitar (Q78987) vs. trumpet (Q8338): 0.0
(Similitud) flute (Q11405) vs. trumpet (Q8338): 0.4


Tiene sentido que el piano se parezca un poco a la guitarra y a la guitarra eléctrica porque los 3 tienen cuerdas, mientras que no tienen nada que ver con una flauta o una trompeta porque estos últimos no tienen cuerdas. Por otro lado, es razonable que una guitarra y una guitarra eléctrica se parezcan mucho porque ambos son, valga la redundancia, guitarras. Por último, la flauta y la trompeta solo guardan cierta similitud entre ellas por ser los únicos instrumentos de viento.